In [1]:
from datetime import datetime
import requests
from time import sleep
from requests import ConnectTimeout, ReadTimeout
import pandas as pd
import plotly.graph_objects as go

In [2]:
def ajustar_data(df):
    meses = {'Jan': '01', 'Fev': '02', 'Mar': '03', 'Abr': '04', 'Mai': '05', 'Jun': '06',
             'Jul': '07', 'Ago': '08', 'Set': '09', 'Out': '10', 'Nov': '11', 'Dez': '12'}

    for mes, numero in meses.items():
        df = df.str.replace(mes, numero)

    df = df.str.replace(" ", "/")
    df = pd.to_datetime(df, format="%d/%m/%Y")
    return df
    
def buscar_proximas_series(anos=2):
    nomes_series = []
    data_atual = datetime.today()
    mes_atual = data_atual.month
    ano_atual = data_atual.year
    meses = {'1': 'F', '2': 'G', '3': 'H', '4': 'J', '5': 'K', '6': 'M',
             '7': 'N', '8': 'Q', '9': 'U', '10': 'V', '11': 'X', '12': 'Z'}

    for i, letra in meses.items():
        if int(i) >= int(mes_atual):
            nomes_series.append("DI1" + letra + str(ano_atual)[2:4])

    for i in range(ano_atual + 1, ano_atual + anos + 1):
        ano = str(i)
        nomes_series.append("DI1F" + ano[2:4])

    return nomes_series

def buscar_dados_futuros(titulo):
    sleep(0.1)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    url = "https://br.advfn.com/bolsa-de-valores/bmf/{}/historico/mais-dados-historicos".format(
        titulo)
    print("Abrindo " + url)
    html = requests.get(url, headers=headers, verify=False, timeout=5)
    # print(html.text)
    dif = pd.read_html(html.text, decimal=',', thousands='.')[1]
    dif = dif[["Data", "Fechamento"]]
    dif = dif.rename(columns={"Fechamento": titulo})
    dif[['Data']] = dif[['Data']].apply(lambda x: ajustar_data(x))
    dif = dif.set_index("Data")
    return dif

In [3]:
anos = 7 #@param {type:"integer"}
semanas = 7 #@param {type:"integer"}

In [4]:
series = []
nomes_series = buscar_proximas_series(anos)
print(nomes_series)

for nome_serie in nomes_series:
    try:
        s = buscar_dados_futuros(nome_serie)
        s = s.loc[~s.index.duplicated(keep='first')]
        series.append(s)
    except ConnectTimeout:
        print("Timeout ao buscar dados de " + nome_serie + ". Buscando próxima série...")
    except ReadTimeout:
        print("Timeout ao buscar dados de " + nome_serie + ". Buscando próxima série...")

['DI1Z22', 'DI1F23', 'DI1F24', 'DI1F25', 'DI1F26', 'DI1F27', 'DI1F28', 'DI1F29']
Abrindo https://br.advfn.com/bolsa-de-valores/bmf/DI1Z22/historico/mais-dados-historicos
/shared-libs/python3.9/py/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'br.advfn.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
Abrindo https://br.advfn.com/bolsa-de-valores/bmf/DI1F23/historico/mais-dados-historicos
/shared-libs/python3.9/py/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'br.advfn.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
Abrindo https://br.advfn.com/bolsa-de-valores/bmf/DI1F24/historico/mais-dados-historicos
/shared-

In [5]:
difuturo = pd.concat(series, axis=1)
difuturo.index = pd.to_datetime(difuturo.index)
difuturo

,DI1Z22,DI1F23,DI1F24,DI1F25,DI1F26,DI1F27,DI1F28,DI1F29
Data,,,,,,,,
2022-12-12,13.654,13.656,13.880,13.300,13.115,13.050,13.04,13.10
2022-12-09,13.654,13.656,13.825,13.130,12.945,12.880,12.89,12.92
2022-12-08,13.654,13.656,13.830,13.060,12.865,12.805,12.79,12.85
2022-12-07,13.654,13.666,13.850,13.025,12.780,12.695,12.66,12.73
2022-12-06,13.654,13.666,13.925,13.075,12.815,12.695,12.66,12.68
...,...,...,...,...,...,...,...,...
2022-09-20,13.764,13.770,13.215,11.880,11.600,11.525,11.55,11.63
2022-09-19,13.770,13.770,13.240,11.930,11.655,11.570,11.59,11.68
2022-09-16,13.776,13.795,13.230,12.025,11.765,11.710,11.73,11.81


In [6]:
dayofweek = difuturo.index.dayofweek
semanal = difuturo.iloc[(dayofweek == 0)]
difuturo = semanal.head(semanas)
difuturo = difuturo.T

layout = go.Layout(
    annotations=[
        dict(
            x=1.12,
            y=1.05,
            align="right",
            valign="top",
            text='Semanas:',
            showarrow=False,
            xref="paper",
            yref="paper",
            xanchor="center",
            yanchor="top"
        )
    ]
)
fig = go.Figure(layout=layout)

#print(difuturo)

fig.update_layout(title_text="Curva de Juros (dados extraídos da ADVFN)",
                  title_font_size=30)

fig.update_layout(
    autosize=False,
    width=1000,
    height=600)

fig.update_xaxes(title_text="Prazos ou vencimentos")
fig.update_yaxes(title_text="Taxas")

for numero, i in enumerate(difuturo):
    fig.add_trace(go.Scatter(x=difuturo.index, y=difuturo[i], mode='lines',
                              name="Semana " + str(numero + 1) + ": " + i.strftime(
                                  '%d/%m/%Y')))
    
fig

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cf6f5887-9730-4d73-8b48-f465b0268709' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>